In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from __future__ import print_function
import time
import pafy
from bs4 import BeautifulSoup
import requests
import re
import webbrowser
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Image
from IPython.core.display import HTML 
import time
from ipywidgets import FloatProgress
from IPython.display import display
from textblob import TextBlob
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from IPython.display import Image
from nltk.tokenize import RegexpTokenizer

/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [60]:
df = pd.read_csv('new_search.csv', index_col=0)
df2 = pd.read_csv('current_search.csv', index_col=0)

In [61]:
df = df.drop(['polarity','length','article_similarity', 'source_similarity','cosine_similarity','subject_matter','cluster'], axis =1)

In [62]:
def df_all_articles(articles_df):
    articles_df['text'] = np.nan
    for i in range(0, len(articles_df)):
        if articles_df['source_id'][i] == 'the-huffington-post':
            pass
        else:
            html_page = requests.get(articles_df['url'][i]) 
            soup = BeautifulSoup(html_page.content, 'html.parser', from_encoding="iso-8859-1")
            article = soup.findAll('p')
            str_list = []
            list_ = []
            for j in range(0,len(article)):
                clean = article[j].text
                str_list.append(clean)
            article_ = ' '.join(str_list)
            item = article_.encode("ascii", "ignore")
            clean_article = item.decode("utf-8")
            clean_article = clean_article.strip('\n')
            articles_df['text'][i] = clean_article
    return articles_df

In [88]:
# df_all_articles(df)

In [64]:
df = df.dropna()

In [65]:
df = df.reset_index()
df = df.drop('index',axis = 1)

In [68]:
df = df[(df.source_id != 'abc-news')]
df = df.reset_index()

In [70]:
df = df.drop('index',axis = 1)

In [46]:
# df = df.drop('level_0',axis = 1)

In [72]:
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

def df_with_polarity_score(art_df):
    art_df['neg'] = np.nan
    art_df['neu'] = np.nan
    art_df['pos'] = np.nan
    art_df['compound'] = np.nan
    art_df = art_df.reset_index(drop=True)
    for i in range(0, len(art_df)):
        scores = sid.polarity_scores(art_df['text'][i])
        art_df['neg'][i] = scores['neg']
        art_df['neu'][i] = scores['neu']
        art_df['pos'][i] = scores['pos']
        art_df['compound'][i] = scores['compound']
    return art_df

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/markrethana/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [73]:
# from sklearn.feature_extraction.text import CountVectorizer
# cvec = CountVectorizer()
# cvec.fit(my_df.text)
# neg_doc_matrix = cvec.transform(my_df[my_df.target == 0].text)
# pos_doc_matrix = cvec.transform(my_df[my_df.target == 1].text)
# neg_tf = np.sum(neg_doc_matrix,axis=0)
# pos_tf = np.sum(pos_doc_matrix,axis=0)
# neg = np.squeeze(np.asarray(neg_tf))
# pos = np.squeeze(np.asarray(pos_tf))
# term_freq_df = pd.DataFrame([neg,pos],columns=cvec.get_feature_names()).transpose()

In [74]:
def df_with_subjectivity(df):
    df['subjectivity'] = np.nan
    for i in range(0, len(df)):
        analysis = TextBlob((df['text'][i]))
        df['subjectivity'][i] =  analysis.sentiment.subjectivity
    return df
                            

In [87]:
# final_df = df_with_polarity_score(df)

In [86]:
# final_df = df_with_subjectivity(final_df)

In [77]:
def get_vocab(df):
    tokenizer = RegexpTokenizer('[a-z]\w+')
    all_text = []
    for blob in df.text:
        new_blob = blob.lower()
        all_text.append(new_blob)
    corpus = '-'.join(all_text)
    token = tokenizer.tokenize(corpus)
    vocab = sorted(set(token))
    return vocab

In [79]:
def word_metrics(vocab):
    all_metrics = []
    for word in vocab:
        analysis = TextBlob(word)
        metric = {'word':word,'subjectivity':analysis.sentiment.subjectivity,'polarity':analysis.sentiment.polarity}
        all_metrics.append(metric)
    return pd.DataFrame(all_metrics)

In [ ]:
# html_page = requests.get('https://www.youtube.com/results?search_query='+split[0]+'+'+'+'+split[1])
# soup = BeautifulSoup(html_page.content, 'html.parser', from_encoding='utf-8')
# videos = soup.findAll('a')

In [85]:
# final_df.to_csv('bk_corpus.csv')

# Corpus Exploration

In [93]:
corpus_df = pd.read_csv('bk_corpus.csv', index_col=0)

In [107]:
corpus_df.medium = corpus_df.medium.fillna('text')

In [116]:
corpus_df = corpus_df.dropna()
# corpus_df = corpus_df.reset_index()
corpus_df = corpus_df.drop('index',axis = 1)

In [132]:
corpus_df['overall_sent'] = np.where(corpus_df['compound']>=0, 'positive', 'negative')

In [142]:
all_words = get_vocab(corpus_df)

In [147]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer(token_pattern=r'[A-Za-z]\w+',)
cvec.fit(corpus_df.text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='[A-Za-z]\\w+', tokenizer=None,
        vocabulary=None)

In [170]:
neg_doc_matrix = cvec.transform(corpus_df[corpus_df.overall_sent == 'negative'].text)
pos_doc_matrix = cvec.transform(corpus_df[corpus_df.overall_sent == 'positive'].text)
neg_tf = np.sum(neg_doc_matrix,axis=0)
pos_tf = np.sum(pos_doc_matrix,axis=0)
neg = np.squeeze(np.asarray(neg_tf))
pos = np.squeeze(np.asarray(pos_tf))
term_freq_df = pd.DataFrame([neg,pos],columns=cvec.get_feature_names()).transpose()
term_freq_df = term_freq_df.rename({0:'negative',1:'positive'}, axis='columns')

In [175]:
# term_freq_df.sort_values(by=['negative'], ascending=False)

In [240]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import multiprocessing
import gensim.models.doc2vec
# cores = multiprocessing.cpu_count()

In [183]:
for i, d in enumerate(data):
    print(i)

0
1
2
3


In [225]:
data = list(corpus_df.text)
sources = list(corpus_df.source_name)

tagged_data = [TaggedDocument(words=word_tokenize(word.lower()), tags=[sources[index]]) for index, word in enumerate(data)]
    

In [226]:
tagged_data[0]

TaggedDocument(words=['the', 'senate', 'confirmation', 'hearing', 'for', 'brett', 'kavanaugh', ',', 'president', 'donald', 'trump', "'s", 'u.s.', 'supreme', 'court', 'pick', ',', 'descended', 'into', 'chaos', 'on', 'tuesday', ',', 'as', 'democrats', 'protested', 'about', 'republicans', 'blocking', 'access', 'to', 'documents', 'concerning', 'the', 'nominee', "'s", 'white', 'house', 'work', 'more', 'than', 'a', 'decade', 'ago', '.', 'democratic', 'senators', 'repeatedly', 'interrupted', 'the', 'judiciary', 'committee', "'s", 'republican', 'chairman', 'chuck', 'grassley', 'at', 'the', 'outset', 'of', 'the', 'hearing', 'and', 'dozens', 'of', 'shouting', 'protesters', 'were', 'removed', 'one', 'by', 'one', 'by', 'security', 'personnel', '.', 'some', 'seven', 'hours', 'after', 'the', 'hearing', 'began', ',', 'kavanaugh', 'the', 'conservative', 'federal', 'appeals', 'court', 'judge', 'picked', 'by', 'trump', 'for', 'a', 'lifetime', 'job', 'on', 'the', 'top', 'u.s.', 'judicial', 'body', 'final

In [254]:
assert gensim.models.doc2vec.FAST_VERSION > -1
max_epochs = 200
vec_size = 100
alpha = 0.025
cores = multiprocessing.cpu_count()

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=2,
                dm =0, workers=cores)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")


/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [253]:
model.iter

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


5

In [257]:
model.most_similar('democrat')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('63', 0.34307003021240234),
 ('thoughts', 0.3347507119178772),
 ('antipathy', 0.333181232213974),
 ('banning', 0.33054009079933167),
 ('introduce', 0.324016273021698),
 ('angus', 0.3207557201385498),
 ('tossup', 0.31640058755874634),
 ('mostly', 0.3114963471889496),
 ('rooted', 0.3056873679161072),
 ('consist', 0.3020482361316681)]

In [258]:
# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('The New York Times')
print(similar_doc)



# print(model.docvecs['1']) #print doc vector

[('Time', 0.7444280385971069), ('BBC News', 0.7008002400398254), ('Business Insider', 0.6946393847465515), ('The Telegraph', 0.6722674369812012), ('Vice News', 0.6688388586044312), ('ABC News', 0.6360713839530945), ('The Guardian (AU)', 0.6355329751968384), ('NBC News', 0.6308814287185669), ('CNN', 0.6269000172615051), ('New York Magazine', 0.6130892038345337)]


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [259]:
similar_doc = model.docvecs.most_similar('Time')
print(similar_doc)

[('BBC News', 0.8105869293212891), ('Business Insider', 0.8018974661827087), ('The Telegraph', 0.7758575081825256), ('Vice News', 0.769325315952301), ('ABC News', 0.7505406141281128), ('The New York Times', 0.7444280385971069), ('NBC News', 0.7301639914512634), ('The Guardian (AU)', 0.7238157987594604), ('New York Magazine', 0.7086154222488403), ('Mashable', 0.6647524237632751)]


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [260]:
similar_doc = model.docvecs.most_similar('Fox News')
print(similar_doc)

[('National Review', 0.7998780608177185), ('USA Today', 0.7808328866958618), ('CBC News', 0.7581006288528442), ('The Washington Times', -0.608645498752594), ('Reuters', -0.6432740092277527), ('CNN', -0.644241988658905), ('Mashable', -0.6445731520652771), ('The Washington Post', -0.6553245186805725), ('CBS News', -0.6608672738075256), ('Associated Press', -0.6701783537864685)]


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [216]:
list(corpus_df.source_name)[113]

'USA Today'